### Proof of likelihood function nonconvexity
This script identifies a situation in which the likelihood function is nonconvex. Specifically, let 

${\mathbf{D}_D} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}$ and $ {\mathbf{D}_I} = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$

Then, define our guess matrix $\widehat{\mathbf{D}}$ to be 

$\widehat{\mathbf{D}} = \begin{bmatrix} w & x \\ y & z \end{bmatrix}$

We can know write the likelihood function of our guess explicitly using the `sympy` package.

In [1]:
from sympy import *
var = ["w", "x","y","z"]
from IPython.display import display

w, x, y, z = symbols(" ".join(var))

# Define our likelihood function
fun = (w/(w+x)-1)**2  + (w/(w+y))**2 
fun += (x/(w+x))**2   + (x/(x+z))**2
fun += (y/(y+z))**2   + (y/(w+y)-1)**2
fun += (z/(y+z)-1)**2 + (z/(x+z))**2

# Switch to negative log-likelihood so we can minimize
fun *= 1

The Jacobian matrix of this function is really nastly looking:

In [2]:
jac = Matrix([[fun.diff(a,b) for a in var] for b in var])
display(jac)

Matrix([
[2*(3*w**2/(w + y)**4 - 4*w/(w + y)**3 + 3*x**2/(w + x)**4 + y**2/(w + y)**4 + 2*y*(y/(w + y) - 1)/(w + y)**3 + (w + y)**(-2) + 3*(w/(w + x) - 1)**2/(w + x)**2),                                                                                       2*(3*w*(w/(w + x) - 1)/(w + x) - w/(w + x) + 3*x**2/(w + x)**2 - 2*x/(w + x) + 1)/(w + x)**2,                                                  2*(3*w**2/(w + y)**2 - 2*w/(w + y) + y*(y/(w + y) - 1)/(w + y) + (y/(w + y) - 1)*(2*y/(w + y) - 1))/(w + y)**2,                                                                                                                                0],
[                                                 2*(w*(w/(w + x) - 1)/(w + x) + 3*x**2/(w + x)**2 - 2*x/(w + x) + (w/(w + x) - 1)*(2*w/(w + x) - 1))/(w + x)**2, 2*(w**2/(w + x)**4 + 2*w*(w/(w + x) - 1)/(w + x)**3 + 3*x**2/(x + z)**4 + 3*x**2/(w + x)**4 - 4*x/(x + z)**3 - 4*x/(w + x)**3 + 3*z**2/(x + z)**4 + (x + z)**(-2) + (w + x)**(-2)),                

It doesn't appear that this matrix should be positive semidefinite for all $w, x, y, z$ combos, so we iteratively find a counterexample:

In [3]:
r = [(i+1)*0.01 for i in range(0,99)]

def search():
    for i in r:
        for j in r:
            for k in r:
                for l in r:
                    mat = jac.subs([(w,i),(x,j),(y,k),(z,l)])
                    for eig in mat.eigenvals().keys():
                        val = eig.evalf()
                        if im(val) > 0 or re(val) < 0:
                            print "Not convex! " + str(val)
                            print(i,j,k,l)
                            return
search()

Not convex! -1830.12701892219 + 2.23748904465343e-27*I
(0.01, 0.01, 0.01, 0.01)
